In [ ]:
from itertools import chain

import anndata as ad
import itertools
import pandas as pd
import scanpy as sc
import sckade
import seaborn as sns
from matplotlib import rcParams

In [2]:
sckade.plot.set_publication_params()
rcParams["figure.figsize"] = (4, 4)

In [3]:
rna = ad.read_h5ad("./data/preprocess/10x-rna-pp.h5ad")
atac = ad.read_h5ad("./data/preprocess/10x-atac-pp.h5ad")

In [ ]:
rna.layers

In [5]:
sckade.models.configure_dataset(
    rna, use_layer="counts", use_rep="X_pca"
)

In [6]:
sckade.models.configure_dataset(
    atac, use_rep="X_lsi"
)

In [ ]:
kade = sckade.models.fit_sckade(
    {"rna": rna, "atac": atac},
    fit_kws={"directory": "kade"}
)

In [8]:
kade.save("kade.dill")

In [9]:
rna.obsm["X_kade"] = kade.encode_data("rna", rna)
atac.obsm["X_kade"] = kade.encode_data("atac", atac)

In [ ]:
combined = ad.concat([rna, atac])
sc.pp.neighbors(combined, use_rep="X_kade", metric="cosine")
sc.tl.umap(combined)
sc.pl.umap(combined, color=["cell_type", "domain"], wspace=0.65)

In [11]:
rna.write("./data/latent/10x-rna-emb.h5ad", compression="gzip")
atac.write("./data/latent/10x-atac-emb.h5ad", compression="gzip")

In [12]:
rna = ad.read_h5ad("./data/latent/10x-rna-emb.h5ad")
atac = ad.read_h5ad("./data/latent/10x-atac-emb.h5ad")

In [13]:
rna, atac

In [14]:
rna_latent = pd.DataFrame(rna.obsm['X_kade'])
atac_latent = pd.DataFrame(atac.obsm['X_kade'])

In [15]:
rna_latent.to_csv("./data/latent/10x_rna_latent.csv", index = False, header = None)
atac_latent.to_csv("./data/latent/10x_atac_latent.csv", index=False, header = None)